# Recommendations Systems: Embeddings

Credits to: <a href="https://www.tensorflow.org/tutorials/word2vec" style="color: #6D00FF;">Google</a>

Image and audio processing systems work with rich, high-dimensional datasets encoded as vectors of the individual raw pixel-intensities for image data, or e.g. power spectral density coefficients for audio data. For tasks like object or speech recognition we know that all the information required to successfully perform the task is encoded in the data (because humans can perform these tasks from the raw data). However, natural language processing systems traditionally treat words as discrete atomic symbols, and therefore 'cat' may be represented as Id537 and 'dog' as Id143. These encodings are arbitrary, and provide no useful information to the system regarding the relationships that may exist between the individual symbols. This means that the model can leverage very little of what it has learned about 'cats' when it is processing data about 'dogs' (such that they are both animals, four-legged, pets, etc.). Representing words as unique, discrete ids furthermore leads to data sparsity, and usually means that we may need more data in order to successfully train statistical models. Using vector representations can overcome some of these obstacles.

![Image of Yaktocat](images/embeddings.png)

Vector space models (VSMs) represent (embed) words in a continuous vector space where semantically similar words are mapped to nearby points ("are embedded nearby each other"). VSMs have a long, rich history in NLP, but all methods depend in some way or another on the Distributional Hypothesis, which states that words that appear in the same contexts share semantic meaning. The different approaches that leverage this principle can be divided into two categories: count-based methods (e.g. Latent Semantic Analysis), and predictive methods (e.g. neural probabilistic language models).

This distinction is elaborated in much more detail by <a href="http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf" style="color: #6D00FF;">Baroni et al</a>, but in a nutshell: Count-based methods compute the statistics of how often some word co-occurs with its neighbor words in a large text corpus, and then map these count-statistics down to a small, dense vector for each word. Predictive models directly try to predict a word from its neighbors in terms of learned small, dense embedding vectors (considered parameters of the model).

Word2vec is a particularly computationally-efficient predictive model for learning word embeddings from raw text. It comes in two flavors, the Continuous Bag-of-Words model (CBOW) and the Skip-Gram model (Section 3.1 and 3.2 in  <a href="https://arxiv.org/pdf/1301.3781.pdf" style="color: #6D00FF;">Mikolov et al.</a>). Algorithmically, these models are similar, except that CBOW predicts target words (e.g. 'mat') from source context words ('the cat sits on the'), while the skip-gram does the inverse and predicts source context-words from the target words. This inversion might seem like an arbitrary choice, but statistically it has the effect that CBOW smoothes over a lot of the distributional information (by treating an entire context as one observation). For the most part, this turns out to be a useful thing for smaller datasets. However, skip-gram treats each context-target pair as a new observation, and this tends to do better when we have larger datasets. We will focus on the skip-gram model for the rest of this tutorial.


### But what does it have to do with Recommender Systems?

The way that teams usually employ embeddings in recommender systems is to understand the most relevant aspects of a data distribution. This understanding can be used in case there is a significant data shift. Let's take the example of Twitter. Every day, hundreds of millions of new Tweets flow through Twitter. Models that attempt to extract findings from this incredible ammount of information must face the rapid covariate shift that is endemic to the platform. As you can see in Figures 1 and 2; the correlation between words change over time and moreover the apperance of words in the platform varies with time. An additional It is important to note that embedding layers can be used to deal with the sparse matrix problem in recommender systems. 

![Image of Yaktocat](images/twitter.png)

Recommender systems are being used everywhere and you are probably being influenced by them every day. The most common examples are Amazon’s product recommendation and Netflix’s program recommendation systems. Netflix actually held a $1,000,000 challenge to find the best collaborative filtering algorithm for their recommender system. 

As we mentioned in the first part of this tutorial, there are two main types of recommender systems. Just as a reminder:

- Content-based filtering: This type of filtering is based on data about the item/product. For example, we have our users fill out a survey on what movies they like. If they say that they like sci-fi movies we recommend them sci-fi movies. In this case al lot of meta-information has to be available for all items.

- Collaborative filtering: Let’s find other people like you, see what they liked and assume you like the same things. People like you = people who rated movies that you watched in a similar way. In a large dataset this has proven to work a lot better than the meta-data approach. Essentially asking people about their behavior is less good compared to looking at their actual behavior. Discussing this further is something for the psychologists among us.
In order to solve this problem we can create a huge matrix of the ratings of all users against all movies. However, in many cases this will create an extremely sparse matrix. Just think of your Netflix account. What percentage of their total supply of series and movies have you watched? It’s probably a pretty small percentage. Then, through gradient descent we can train a neural network to predict how high each user would rate each movie. Let me know if you would like to know more about the use of deep learning in recommender systems and we can explore it further together. In conclusion, embedding layers are amazing and should not be overlooked.

With that in mind let's get started with this last session!

## 0. Importing all the things

Let's import all we will need for this tutorial first so we don't run into any surprises midway.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import sys
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from tensorflow.contrib.tensorboard.plugins import projector

## 1. Get Data

The data for this tutorial can be found in the repository. It is saved in zip format.  We are now unzipping it in order to get the be able to form our embeddings. The dataset comes from http://mattmahoney.net/dc/textdata.html and is used for embeddings benchmarks. The data is UTF-8 encoded XML consisting primarily of English text. With that in mind, let's get started.

In [ ]:
with zipfile.ZipFile('text8.zip') as f:
    vocabulary = tf.compat.as_str(f.read(f.namelist()[0])).split()

## 2. Data and vocabulary

Now that we've read in the raw text and converting it into a list of strings, we'll need to convert this list into a dictionary of (input, output) pairs as described above for the Skip-Gram model. We'll also replace rare words in the dictionary with the token UNK, as is standard in this kind of NLP task. 'UNK' in our dataset can be interpreted as unknown or out-of-the-vocabulary word. We see that there is 17005207 words in vocabulary.

In [ ]:
print("The first 5 elements of the dataset are:", vocabulary[:5])
print("The length of the dataset is", len(vocabulary))

Let's now change the vocabulary_size to something more managable and then build our dataset. We are very close to getting ready to build our embeddings. 
<font color='red'>(##TODO: Create the reversed dictionary, this is just a dictionary where the keys of the `dictionary` variable are the values and the values are the keys (just justifying the reversed name).</font>

In [ ]:
vocabulary_size = 50000
count = [['UNK', -1]]
count.extend(collections.Counter(vocabulary).most_common(vocabulary_size - 1))
dictionary = dict()
for word, _ in count:
  dictionary[word] = len(dictionary)
data = list()
unk_count = 0
for word in vocabulary:
  index = dictionary.get(word, 0)
  if index == 0:  # dictionary['UNK']
    unk_count += 1
  data.append(index)
count[0][1] = unk_count
reverse_dictionary = ##YOUR CODE HERE

Let's look at the most common words used in the dictionary. Is it even a surprise that most of them are prepositions?

In [ ]:
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

## 3. Generate Batch

This function is used to generate training batch for the skip-gram model. We are using minibatch stochastic-gradient descent to optimize the objective function. For this, we generate a training batch based on following input parameters: batch_size, num_samples, and skip_window.

- batch_size is used to control the size of the minibatch* of training data. i.e.,* it is the number of pairs (CENTER_WORD, CONTEXT_WORD). Note that all CENTER_WORDs are stored and returned in the list batch, and all CONTEXT_WORDs are stored and returned in the list label.

- In the skip-gram model, we generate training data by capturing the context words from the surrondings of a target word. Here the parameter: skip_window defines a sliding window size of size (2*skip_window+1), i.e., up to skip_window words to the left and right of the center word.

- Within a sliding window, we need to sample num_samples of context words (i.e., excluding the center word). num_samples should be no larger than 2*skip_window.

Example:
As an example, with batch_size= 8, skip_window = 1, and num_samples = 2, it generates following data batch. We consider these words as (input,output) pairs for the skip-gram model.

Text: anarchism originated as a term of abuse first used against early 

(CENTER_WORD, CONTEXT_WORD) Pairs:
(originated,anarchism), (originated,as), (as,originated), (as,a), (a,term) (a,as), (term,of), (term,a)

<font color='red'>(##TODO: Try running with different `batch_sizes`, `skip_windows` and `num_samples` and seeing the result).</font>

In [ ]:
data_index = 0
batch_size = 8
num_skips = 2
skip_window = 1

<font color='red'>##TODO: For the buffer we want to make sure that we are able easily append and pop values from either side of the container. Any ideas of what data structure can be used here? ##HINT: You may find this <a href="https://docs.python.org/2/library/collections.html">this</a> helpful.</font>

In [ ]:
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 
  buffer = ##YOUR CODE HERE 
  if data_index + span > len(data):
    data_index = 0
  buffer.extend(data[data_index:data_index + span])
  data_index += span
  for i in range(batch_size // num_skips):
    context_words = [w for w in range(span) if w != skip_window]
    words_to_use = random.sample(context_words, num_skips)
    for j, context_word in enumerate(words_to_use):
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[context_word]
    if data_index == len(data):
      buffer.extend(data[0:span])
      data_index = span
    else:
      buffer.append(data[data_index])
      data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

Let's now visualize it. <font color='red'> ##TODO: Call the generate_batch function in order to generate the batch and labels. </font>

In [ ]:
batch, labels = generate_batch(batch_size=batch_size, num_skips=num_skips, skip_window=skip_window)
for i in range(8):
  print(batch[i], reversed_dictionary[batch[i]], '--', labels[i, 0],
        reversed_dictionary[labels[i, 0]])

## 4. Build and Train the Skip-Gram

In [ ]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.
num_sampled = 64  # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

This is all about embeddings, so let's define our embedding matrix. This is just a big random matrix to start. We'll initialize the values to be uniform in the unit cube. The noise-contrastive estimation loss is defined in terms of a logistic regression model. For this, we need to define the weights and biases for each word in the vocabulary (also called the output weights as opposed to the input embeddings). Now that we have the parameters in place, we can define our skip-gram model graph. For simplicity, let's suppose we've already integerized our text corpus with a vocabulary so that each word is represented as an integer. The skip-gram model takes a batch full of integers representing the source context words and the the target words. This will allow you to train the skipgram model. Once you are done doing that we can finally work on training. To learn more about NCE loss in skip-gram models, click <a href=" https://mk-minchul.github.io/NCE/" style="color: #6D00FF;">here</a>.

- <font color='red'> ##TODO: [1] Write code to loop up the embeddings for `train_inputs`. Remember to define a `name_scope`. and to define embeddings as a `tf.Variable`. then we can use the `tf.nn.embedding_lookup` in order to be able to look them up</font>
- <font color='red'> ##TODO: [2] Construct the SGD optimizer using a learning rate of 1.0. Remember that the optimizer would like to minimize in the loss. </font>
- <font color='red'> ##TODO: [3] Add a variable initializer </font>

In [ ]:
graph = tf.Graph()
with graph.as_default():

  # Input data.
  with tf.name_scope('inputs'):
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    with tf.name_scope('embeddings'):
     ## [1] YOUR CODE

    # Construct the variables for the NCE loss
    with tf.name_scope('weights'):
      nce_weights = tf.Variable(
          tf.truncated_normal(
              [vocabulary_size, embedding_size],
              stddev=1.0 / math.sqrt(embedding_size)))
    with tf.name_scope('biases'):
      nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  with tf.name_scope('loss'):
    loss = tf.reduce_mean(
        tf.nn.nce_loss(
            weights=nce_weights,
            biases=nce_biases,
            labels=train_labels,
            inputs=embed,
            num_sampled=num_sampled,
            num_classes=vocabulary_size))

  # Add the loss value as a scalar to summary.
  tf.summary.scalar('loss', loss)
    
 # Construct the optimizer
  with tf.name_scope('optimizer'):
    optimizer = ## [2] YOUR CODE HERE

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
                                            valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Merge all summaries.
  merged = tf.summary.merge_all()

  # Add variable initializer.
  init = # [3] YOUR CODE HERE

  # Create a saver.
  saver = tf.train.Saver()

## 4. Training

In order to execute the model, we initialize a session object using tf.Session(), and call respective node via session.run() or eval(). General workflow for the training process is as under:

- Define number of training steps.
- Initialize all variables, i.e, embeddings, weights and biases using session.run(init).
- Placeholder, (train_inputs, and train_labels) are used to feed input data to skip-gram using the method generate_batch
optimizer, and loss are executed by calling session.run()
- Print out average loss after every 5000 iterations
- Evaluate the similarity after every 10000 iterations. Look for top-10 nearest neighbours for words in sample set.


<font color='red'> ##TODO: Add logic to look for top-10 nearest neighbours for words in sample set. You may need to use the `reversed_dictionary` and `argsort`. While debugging this, you may want to reduce `num_steps` for faster iteration. </font>

In [ ]:
num_steps = 100001
log_dir = 'logs'

del vocabulary

with tf.Session(graph=graph) as session:
  # Open a writer to write summaries.
  writer = tf.summary.FileWriter(log_dir, session.graph)

  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(batch_size, num_skips,
                                                skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # Define metadata variable.
    run_metadata = tf.RunMetadata()

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    # Also, evaluate the merged op to get all summaries from the returned "summary" variable.
    # Feed metadata variable to session for visualizing the graph in TensorBoard.
    _, summary, loss_val = session.run(
        [optimizer, merged, loss],
        feed_dict=feed_dict,
        run_metadata=run_metadata)
    average_loss += loss_val

    # Add returned summaries to writer in each step.
    writer.add_summary(summary, step)
    # Add metadata to visualize the graph for the last run.
    if step == (num_steps - 1):
      writer.add_run_metadata(run_metadata, 'step%d' % step)

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        ## YOUR CODE HERE
  final_embeddings = normalized_embeddings.eval()

  # Write corresponding labels for the embeddings.
  with open(log_dir + '/metadata.tsv', 'w') as f:
    for i in xrange(vocabulary_size):
      f.write(reversed_dictionary[i] + '\n')

  # Save the model for checkpoints.
  saver.save(session, os.path.join(log_dir, 'model.ckpt'))

  # Create a configuration for visualizing embeddings with the labels in TensorBoard.
  config = projector.ProjectorConfig()
  embedding_conf = config.embeddings.add()
  embedding_conf.tensor_name = embeddings.name
  embedding_conf.metadata_path = os.path.join(log_dir, 'metadata.tsv')
  projector.visualize_embeddings(writer, config)

writer.close()

## 5. Visualization of Embeddings

After training has finished we can visualize the learned embeddings using t-SNE. t-SNE is a tool to visualize high-dimensional data. It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. t-SNE has a cost function that is not convex, i.e. with different initializations we can get different results.


In [ ]:
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')

<font color='red'> ##TODO: Use the `fit_transform` method of TSNE to give you the `low_dim_embs` for the first 500 entries.</font>

In [ ]:
plot_only = 500
low_dim_embs = ## YOUR CODE HERE
labels = [reverse_dictionary[i] for i in xrange(plot_only)]

<font color='red'> ##TODO: Add code to plot the low dimension embeddings. </font>

In [ ]:
# assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
plt.figure(figsize=(18, 18))  # in inches
for i, label in enumerate(labels):
  x, y = low_dim_embs[i, :]
  ## YOUR CODE HERE
  plt.annotate(
      label,
      xy=(x, y),
      xytext=(5, 2),
      textcoords='offset points',
      ha='right',
      va='bottom')

In [ ]:
plt.show()

In this tutorial we covered the word2vec model, a computationally efficient model for learning word embeddings. We motivated why embeddings are useful, discussed efficient training techniques and showed how to implement all of this in TensorFlow. Overall, we hope that this has show-cased how TensorFlow affords you the flexibility you need for early experimentation, and the control you later need for bespoke optimized implementation. Moreover we hope that this is just the stepping stone in your experience with embeddings and recommendation systems.



# References/Credits:
[1] https://towardsdatascience.com/deep-learning-4-embedding-layers-f9a02d55ac12

[2] https://www.tensorflow.org/tutorials/word2vec

[3] https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf

[4] http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html